## Practical 1: word2vec
<p>Oxford CS - Deep NLP 2017<br>
https://www.cs.ox.ac.uk/teaching/courses/2016-2017/dl/</p>
<p>[Yannis Assael, Brendan Shillingford, Chris Dyer]</p>

This practical is presented as an IPython Notebook, with the code written for recent versions of **Python 3**. The code in this practical will not work with Python 2 unless you modify it. If you are using your own Python installation, ensure you have a setup identical to that described in the installation shell script (which is intended for use with the department lab machines). We will be unable to support installation on personal machines due to time constraints, so please use the lab machines and the setup script if you are unfamiliar with how to install Anaconda.

To execute a notebook cell, press `shift-enter`. The return value of the last command will be displayed, if it is not `None`.

Potentially useful library documentation, references, and resources:

* IPython notebooks: <https://ipython.org/ipython-doc/3/notebook/notebook.html#introduction>
* Numpy numerical array library: <https://docs.scipy.org/doc/>
* Gensim's word2vec: <https://radimrehurek.com/gensim/models/word2vec.html>
* Bokeh interactive plots: <http://bokeh.pydata.org/en/latest/> (we provide plotting code here, but click the thumbnails for more examples to copy-paste)
* scikit-learn ML library (aka `sklearn`): <http://scikit-learn.org/stable/documentation.html>
* nltk NLP toolkit: <http://www.nltk.org/>
* tutorial for processing xml in python using `lxml`: <http://lxml.de/tutorial.html> (we did this for you below, but in case you need it in the future)

In [1]:
import numpy as np
import os
from random import shuffle
import re

In [2]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

### Part 0: Download the TED dataset

In [3]:
import urllib.request
import zipfile
import lxml.etree

In [4]:
# Download the dataset if it's not already there: this may take a minute as it is 75MB
if not os.path.isfile('ted_en-20160408.zip'):
    urllib.request.urlretrieve("https://wit3.fbk.eu/get.php?path=XML_releases/xml/ted_en-20160408.zip&filename=ted_en-20160408.zip", filename="ted_en-20160408.zip")

In [5]:
# For now, we're only interested in the subtitle text, so let's extract that from the XML:
with zipfile.ZipFile('ted_en-20160408.zip', 'r') as z:
    doc = lxml.etree.parse(z.open('ted_en-20160408.xml', 'r'))
input_text = '\n'.join(doc.xpath('//content/text()'))
del doc

### Part 1: Preprocessing

In this part, we attempt to clean up the raw subtitles a bit, so that we get only sentences. The following substring shows examples of what we're trying to get rid of. Since it's hard to define precisely what we want to get rid of, we'll just use some simple heuristics.

In [6]:
i = input_text.find("Hyowon Gweon: See this?")
input_text[i-20:i+150]

' baby does.\n(Video) Hyowon Gweon: See this? (Ball squeaks) Did you see that? (Ball squeaks) Cool. See this one? (Ball squeaks) Wow.\nLaura Schulz: Told you. (Laughs)\n(Vide'

Let's start by removing all parenthesized strings using a regex:

In [7]:
input_text_noparens = re.sub(r'\([^)]*\)', '', input_text)

We can verify the same location in the text is now clean as follows. We won't worry about the irregular spaces since we'll later split the text into sentences and tokenize it anyway.

In [8]:
i = input_text_noparens.find("Hyowon Gweon: See this?")
input_text_noparens[i-20:i+150]

"hat the baby does.\n Hyowon Gweon: See this?  Did you see that?  Cool. See this one?  Wow.\nLaura Schulz: Told you. \n HG: See this one?  Hey Clara, this one's for you. You "

Now, let's attempt to remove speakers' names that occur at the beginning of a line, by deleting pieces of the form "`<up to 20 characters>:`", as shown in this example. Of course, this is an imperfect heuristic. 

In [9]:
sentences_strings_ted = []
for line in input_text_noparens.split('\n'):
    m = re.match(r'^(?:(?P<precolon>[^:]{,20}):)?(?P<postcolon>.*)$', line)
    sentences_strings_ted.extend(sent for sent in m.groupdict()['postcolon'].split('.') if sent)

# Uncomment if you need to save some RAM: these strings are about 50MB.
# del input_text, input_text_noparens

# Let's view the first few:
sentences_strings_ted[:5]

["Here are two reasons companies fail: they only do more of the same, or they only do what's new",
 'To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation',
 ' Both are necessary, but it can be too much of a good thing',
 'Consider Facit',
 " I'm actually old enough to remember them"]

Now that we have sentences, we're ready to tokenize each of them into words. This tokenization is imperfect, of course. For instance, how many tokens is "can't", and where/how do we split it? We'll take the simplest naive approach of splitting on spaces. Before splitting, we remove non-alphanumeric characters, such as punctuation. You may want to consider the following question: why do we replace these characters with spaces rather than deleting them? Think of a case where this yields a different answer.

In [10]:
sentences_ted = []
for sent_str in sentences_strings_ted:
    tokens = re.sub(r"[^a-z0-9]+", " ", sent_str.lower()).split()
    sentences_ted.append(tokens)

Two sample processed sentences:

In [11]:
len(sentences_ted)

266694

In [12]:
print(sentences_ted[0])
print(sentences_ted[1])

['here', 'are', 'two', 'reasons', 'companies', 'fail', 'they', 'only', 'do', 'more', 'of', 'the', 'same', 'or', 'they', 'only', 'do', 'what', 's', 'new']
['to', 'me', 'the', 'real', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', 'exploration', 'and', 'exploitation']


### Part 2: Word Frequencies

If you store the counts of the top 1000 words in a list called `counts_ted_top1000`, the code below will plot the histogram requested in the writeup.

In [13]:
counts = {}
for sentence in sentences_ted:
    for word in sentence:
        if word in counts:
            counts[word] += 1
        else:
            counts[word] = 1   

words_top_ted = sorted(counts, reverse=True, key=counts.__getitem__)
counts_ted_top1000 = [counts[word] for word in words_top_ted[:1000]]

print(words_top_ted[:100])

['the', 'and', 'to', 'of', 'a', 'that', 'i', 'in', 'it', 'you', 'we', 'is', 's', 'this', 'so', 'they', 'was', 'for', 'are', 'have', 'but', 'what', 'on', 'with', 'can', 't', 'about', 'there', 'be', 'as', 'at', 'all', 'not', 'do', 'my', 'one', 're', 'people', 'like', 'if', 'from', 'now', 'our', 'he', 'an', 'just', 'these', 'or', 'when', 'because', 'very', 'me', 'out', 'by', 'them', 'how', 'know', 'up', 'going', 'had', 'more', 'think', 'who', 'were', 'see', 'your', 'their', 'which', 'would', 'here', 'really', 'get', 've', 'then', 'm', 'world', 'us', 'time', 'some', 'has', 'don', 'actually', 'into', 'way', 'where', 'will', 'years', 'things', 'other', 'no', 'could', 'go', 'well', 'want', 'been', 'make', 'right', 'she', 'said', 'something']


Plot distribution of top-1000 words

In [14]:
hist, edges = np.histogram(counts_ted_top1000, density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

### Part 3: Train Word2Vec

In [15]:
from gensim.models import Word2Vec

D:\Programming\Anaconda\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [16]:
model_ted = Word2Vec(sentences=sentences_ted, size=100, min_count=5)
model_ted.train(sentences_ted, total_examples=model_ted.corpus_count, epochs=20)

63431997

### Part 4: Ted Learnt Representations

Finding similar words: (see gensim docs for more functionality of `most_similar`)

In [17]:
model_ted.most_similar("man")

[('woman', 0.7895859479904175),
 ('guy', 0.7318049669265747),
 ('gentleman', 0.6735437512397766),
 ('soldier', 0.6572358012199402),
 ('lady', 0.6545881628990173),
 ('person', 0.6382701992988586),
 ('boy', 0.6305985450744629),
 ('girl', 0.6223523616790771),
 ('rabbi', 0.6167187690734863),
 ('pianist', 0.5954994559288025)]

In [18]:
model_ted.most_similar("computer")

[('software', 0.6238788962364197),
 ('simulation', 0.6197474002838135),
 ('algorithm', 0.6147148609161377),
 ('computers', 0.6057816743850708),
 ('machine', 0.6004705429077148),
 ('interface', 0.5906597375869751),
 ('browser', 0.5851816534996033),
 ('desktop', 0.584074854850769),
 ('mechanical', 0.5840590000152588),
 ('device', 0.5837869644165039)]

In [19]:
model_ted.most_similar(positive=['insect', 'worker'])

[('ant', 0.692992091178894),
 ('orchid', 0.597510576248169),
 ('ngo', 0.5834742784500122),
 ('egg', 0.5606837272644043),
 ('administrator', 0.5579937696456909),
 ('entrepreneur', 0.5485562086105347),
 ('accountant', 0.5478784441947937),
 ('enzyme', 0.5436000823974609),
 ('electroencephalogram', 0.5423248410224915),
 ('addict', 0.5413431525230408)]

#### t-SNE visualization
To use the t-SNE code below, first put a list of the top 1000 words (as strings) into a variable `words_top_ted`. The following code gets the corresponding vectors from the model, assuming it's called `model_ted`:

In [20]:
# This assumes words_top_ted is a list of strings, the top 1000 words

words_top_vec_ted = model_ted[words_top_ted[:1000]]

In [21]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, random_state=0)
words_top_ted_tsne = tsne.fit_transform(words_top_vec_ted)

In [22]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_ted_tsne[:,0],
                                    x2=words_top_ted_tsne[:,1],
                                    names=words_top_ted[:1000]))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

### Part 5: Wiki Learnt Representations

Download dataset

In [23]:
if not os.path.isfile('wikitext-103-raw-v1.zip'):
    urllib.request.urlretrieve("https://s3.amazonaws.com/research.metamind.io/wikitext/wikitext-103-raw-v1.zip", filename="wikitext-103-raw-v1.zip")

In [24]:
with zipfile.ZipFile('wikitext-103-raw-v1.zip', 'r') as z:
    input_text = str(z.open('wikitext-103-raw/wiki.train.raw', 'r').read(), encoding='utf-8') # Thanks Robert Bastian

Preprocess sentences (note that it's important to remove small sentences for performance)

In [25]:
sentences_wiki = []
for line in input_text.split('\n'):
    s = [x for x in line.split('.') if x and len(x.split()) >= 5]
    sentences_wiki.extend(s)
    
for s_i in range(len(sentences_wiki)):
    sentences_wiki[s_i] = re.sub(r"[^a-z0-9]+", " ", sentences_wiki[s_i].lower())
    sentences_wiki[s_i] = re.sub(r'\([^)]*\)', '', sentences_wiki[s_i])
del input_text

In [26]:
# sample 1/5 of the data
shuffle(sentences_wiki)
print(len(sentences_wiki))
sentences_wiki = sentences_wiki[:int(len(sentences_wiki)/5)]
print(len(sentences_wiki))

4267112
853422


Now, repeat all the same steps that you performed above. You should be able to reuse essentially all the code.

In [27]:
sentences_split_wiki=[]
for wr in sentences_wiki:
    sentences_split_wiki.append(wr.split())

In [28]:
counts_wiki = {}
for sentence in sentences_split_wiki:
    for word in sentence:
        if word in counts_wiki:
            counts_wiki[word] += 1
        else:
            counts_wiki[word] = 1
            

words_top_wiki = sorted(counts_wiki, reverse=True, key=counts_wiki.__getitem__)
counts_wiki_top1000 = [counts_wiki[word] for word in words_top_wiki[:1000]]

print(words_top_wiki[:10], counts_wiki_top1000[:10])

['the', 'of', 'and', 'in', 'to', 'a', 'was', 'on', 's', 'as'] [1286865, 548303, 500282, 434688, 398942, 356755, 214913, 154101, 152790, 144701]


In [29]:
hist, edges = np.histogram(counts_wiki_top1000, density=True, bins=100, normed=True)

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="Top-1000 words distribution")
p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555")
show(p)

In [31]:
model_wiki = Word2Vec(sentences=sentences_split_wiki, size=100, min_count=5)
model_wiki.train(sentences_split_wiki, total_examples=model_wiki.corpus_count, epochs=15)

198133813

In [34]:
model_wiki.most_similar("man")

[('woman', 0.7174842953681946),
 ('girl', 0.6514018177986145),
 ('person', 0.6181576251983643),
 ('boy', 0.5799478888511658),
 ('guy', 0.5705775022506714),
 ('someone', 0.5612919926643372),
 ('magician', 0.5578553676605225),
 ('kid', 0.546867847442627),
 ('lad', 0.5437709093093872),
 ('soldier', 0.5415705442428589)]

In [49]:
model_wiki.most_similar("computer")

[('software', 0.7000124454498291),
 ('kinect', 0.6873171329498291),
 ('computers', 0.6702802181243896),
 ('mainframe', 0.6658933758735657),
 ('technology', 0.66304612159729),
 ('handheld', 0.6344120502471924),
 ('3d', 0.6211332082748413),
 ('macintosh', 0.6160797476768494),
 ('computing', 0.6119583249092102),
 ('ibm', 0.6076227426528931)]

In [56]:
model_wiki.most_similar(positive=['holiday', 'celebration'])

[('thanksgiving', 0.7480095028877258),
 ('celebrations', 0.6707433462142944),
 ('homecoming', 0.6670133471488953),
 ('parades', 0.6527904868125916),
 ('parade', 0.6516865491867065),
 ('feast', 0.6360749006271362),
 ('christmas', 0.6225383281707764),
 ('revue', 0.6199454069137573),
 ('banquet', 0.6071924567222595),
 ('holidays', 0.5992181301116943)]

#### t-SNE visualization

In [38]:
# This assumes words_top_wiki is a list of strings, the top 1000 words
words_top_vec_wiki = model_wiki[words_top_wiki[:1000]]

tsne = TSNE(n_components=2, random_state=0)
words_top_wiki_tsne = tsne.fit_transform(words_top_vec_wiki)

In [39]:
p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE for most common words")

source = ColumnDataSource(data=dict(x1=words_top_wiki_tsne[:,0],
                                    x2=words_top_wiki_tsne[:,1],
                                    names=words_top_wiki[:1000]))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)